<a href="https://colab.research.google.com/github/Harjandar/absa-restaurant-sentiment/blob/main/notebooks/ABSA_LSTM_100Pak_Rev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================
# STEP 0: Import required libraries
# ==============================

import pandas as pd  # Data handling


In [ ]:
# ==============================
# STEP 1: Load TRAIN dataset
# ==============================

train_url = "https://raw.githubusercontent.com/Harjandar/absa-restaurant-sentiment/main/data/raw/restaurants_train_single.csv"

train_df = pd.read_csv(train_url)

# Check dataset shape and top rows
print("Total rows in TRAIN dataset:", len(train_df))
train_df.head()


Total rows in TRAIN dataset: 2507


,sentence Id,sentence,aspect_category,aspect_term,from,to,polarity
0,1004293:0,Judging from previous posts this used to be a ...,RESTAURANT#GENERAL,place,51,56,negative
1,1004293:1,"We, there were four of us, arrived at noon - t...",SERVICE#GENERAL,staff,75,80,negative
2,1004293:2,"They never brought us complimentary noodles, i...",SERVICE#GENERAL,NaN,0,0,negative
3,1004293:3,The food was lousy - too sweet or too salty an...,FOOD#QUALITY,food,4,8,negative
4,1004293:3,The food was lousy - too sweet or too salty an...,FOOD#STYLE_OPTIONS,portions,52,60,negative


In [ ]:
# ==============================
# STEP 2: Inspect all aspect categories (no filtering yet)
# ==============================

print("Original aspect categories and counts:")
print(train_df['aspect_category'].value_counts())


Original aspect categories and counts:
aspect_category
FOOD#QUALITY                849
SERVICE#GENERAL             449
RESTAURANT#GENERAL          422
AMBIENCE#GENERAL            255
FOOD#STYLE_OPTIONS          137
RESTAURANT#MISCELLANEOUS     98
FOOD#PRICES                  90
RESTAURANT#PRICES            80
DRINKS#QUALITY               47
DRINKS#STYLE_OPTIONS         32
LOCATION#GENERAL             28
DRINKS#PRICES                20
Name: count, dtype: int64


In [ ]:
# ==============================
# STEP 3: Map old aspect categories to new simplified ones
# ==============================

aspect_mapping = {
    # FOOD related
    'FOOD#QUALITY': 'FOOD',
    'FOOD#STYLE_OPTIONS': 'FOOD',
    'FOOD#PRICES': 'FOOD',

    # SERVICE related
    'SERVICE#GENERAL': 'SERVICE',

    # RESTAURANT related → DELIVERY/OVERALL EXPERIENCE
    'RESTAURANT#GENERAL': 'DELIVERY_OVERALL',
    'RESTAURANT#PRICES': 'DELIVERY_OVERALL',
    'RESTAURANT#MISCELLANEOUS': 'DELIVERY_OVERALL'
}


In [ ]:
# ==============================
# STEP 4: Keep only rows that exist in mapping
# ==============================

train_df = train_df[train_df['aspect_category'].isin(aspect_mapping.keys())]

# Verify
print("Rows after filtering:", len(train_df))
train_df['aspect_category'].value_counts()


Rows after filtering: 2125


,count
aspect_category,
FOOD#QUALITY,849
SERVICE#GENERAL,449
RESTAURANT#GENERAL,422
FOOD#STYLE_OPTIONS,137
RESTAURANT#MISCELLANEOUS,98
FOOD#PRICES,90
RESTAURANT#PRICES,80


In [ ]:
# ==============================
# STEP 5: Add simplified aspect column
# ==============================

train_df['aspect_simplified'] = train_df['aspect_category'].map(aspect_mapping)

# Check
train_df[['aspect_category', 'aspect_simplified']].head()


,aspect_category,aspect_simplified
0,RESTAURANT#GENERAL,DELIVERY_OVERALL
1,SERVICE#GENERAL,SERVICE
2,SERVICE#GENERAL,SERVICE
3,FOOD#QUALITY,FOOD
4,FOOD#STYLE_OPTIONS,FOOD


In [ ]:
# ==============================
# STEP 6: Check for missing values and drop if any
# ==============================

print("Missing values per column:")
print(train_df.isnull().sum())

# Drop missing rows (if any)
train_df = train_df.dropna()

print("Rows after dropping missing:", len(train_df))


Missing values per column:
sentence Id            0
sentence               0
aspect_category        0
aspect_term          591
from                   0
to                     0
polarity               0
aspect_simplified      0
dtype: int64
Rows after dropping missing: 1534


In [ ]:
# Check final columns
train_df.head()


,sentence Id,sentence,aspect_category,aspect_term,from,to,polarity,aspect_simplified
0,1004293:0,Judging from previous posts this used to be a ...,RESTAURANT#GENERAL,place,51,56,negative,DELIVERY_OVERALL
1,1004293:1,"We, there were four of us, arrived at noon - t...",SERVICE#GENERAL,staff,75,80,negative,SERVICE
3,1004293:3,The food was lousy - too sweet or too salty an...,FOOD#QUALITY,food,4,8,negative,FOOD
4,1004293:3,The food was lousy - too sweet or too salty an...,FOOD#STYLE_OPTIONS,portions,52,60,negative,FOOD
6,1004293:5,Avoid this place!,RESTAURANT#GENERAL,place,11,16,negative,DELIVERY_OVERALL


In [ ]:
# ==============================
# STEP 8: Combine sentence + aspect
# ==============================

# Create a new column for model input
train_df['model_input'] = train_df['sentence'] + " [ASP] " + train_df['aspect_simplified']

# Check first few rows
train_df[['sentence', 'aspect_simplified', 'model_input']].head()


,sentence,aspect_simplified,model_input
0,Judging from previous posts this used to be a ...,DELIVERY_OVERALL,Judging from previous posts this used to be a ...
1,"We, there were four of us, arrived at noon - t...",SERVICE,"We, there were four of us, arrived at noon - t..."
3,The food was lousy - too sweet or too salty an...,FOOD,The food was lousy - too sweet or too salty an...
4,The food was lousy - too sweet or too salty an...,FOOD,The food was lousy - too sweet or too salty an...
6,Avoid this place!,DELIVERY_OVERALL,Avoid this place! [ASP] DELIVERY_OVERALL


In [ ]:
# ==============================
# STEP 9: Encode polarity labels
# ==============================

polarity_mapping = {
    'positive': 2,
    'neutral': 1,
    'negative': 0
}

train_df['polarity_encoded'] = train_df['polarity'].map(polarity_mapping)

# Check mapping
train_df[['polarity', 'polarity_encoded']].head()


,polarity,polarity_encoded
0,negative,0
1,negative,0
3,negative,0
4,negative,0
6,negative,0


In [ ]:
# ==============================
# STEP 10: Basic text cleaning
# ==============================
import re
def clean_text(text):
    text = text.lower()                    # lowercase
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()    # remove extra spaces
    return text

# Apply cleaning
train_df['model_input_clean'] = train_df['model_input'].apply(clean_text)

# Check
train_df[['model_input', 'model_input_clean']].head()


,model_input,model_input_clean
0,Judging from previous posts this used to be a ...,judging from previous posts this used to be a ...
1,"We, there were four of us, arrived at noon - t...",we there were four of us arrived at noon the p...
3,The food was lousy - too sweet or too salty an...,the food was lousy too sweet or too salty and ...
4,The food was lousy - too sweet or too salty an...,the food was lousy too sweet or too salty and ...
6,Avoid this place! [ASP] DELIVERY_OVERALL,avoid this place asp deliveryoverall


In [ ]:
# ==============================
# Check polarity distribution BEFORE balancing
# ==============================

print("Polarity distribution before balancing:")
print(train_df['polarity'].value_counts())


Polarity distribution before balancing:
polarity
positive    1044
negative     438
neutral       52
Name: count, dtype: int64


In [ ]:
# ==============================
# STEP 10b: Balance TRAIN dataset
# ==============================

from sklearn.utils import resample

# Separate each class
df_pos = train_df[train_df['polarity'] == 'positive']
df_neg = train_df[train_df['polarity'] == 'negative']
df_neu = train_df[train_df['polarity'] == 'neutral']

# Upsample neutral to match positive
df_neu_up = resample(df_neu,
                     replace=True,
                     n_samples=len(df_pos),   # match positive
                     random_state=42)

# Upsample negative to match positive
df_neg_up = resample(df_neg,
                     replace=True,
                     n_samples=len(df_pos),
                     random_state=42)

# Combine all classes
train_df_balanced = pd.concat([df_pos, df_neg_up, df_neu_up])

# Shuffle
train_df_balanced = train_df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Use balanced dataset for training
train_df = train_df_balanced

# Check distribution after balancing
print("Polarity distribution AFTER balancing:")
print(train_df['polarity'].value_counts())


Polarity distribution AFTER balancing:
polarity
negative    1044
neutral     1044
positive    1044
Name: count, dtype: int64


In [ ]:
# ==============================
# STEP 11: Tokenization
# ==============================

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Hyperparameters
max_words = 5000      # maximum vocabulary size
max_len = 50          # maximum sequence length

# Create tokenizer
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(train_df['model_input_clean'])

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(train_df['model_input_clean'])

# Pad sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')

print("Example tokenized sequence:", X_train_pad[0])
print("Shape of padded sequences:", X_train_pad.shape)


Example tokenized sequence: [ 19  22 210 132  81 501   3  15   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
Shape of padded sequences: (3132, 50)


In [ ]:
# ==============================
# STEP 12: Prepare labels for LSTM
# ==============================

y_train = train_df['polarity_encoded'].values
print("Example labels:", y_train[:10])
print("Shape of labels:", y_train.shape)


Example labels: [0 1 2 2 0 1 1 0 0 2]
Shape of labels: (3132,)


In [ ]:
# ==============================
# STEP 13: Load TEST dataset
# ==============================

test_url = "https://raw.githubusercontent.com/Harjandar/absa-restaurant-sentiment/main/data/raw/restaurants_test_single.csv"

test_df = pd.read_csv(test_url)

print("Total rows in TEST dataset:", len(test_df))
test_df.head()


Total rows in TEST dataset: 859


,sentence Id,sentence,aspect_category,aspect_term,from,to,polarity
0,en_BlueRibbonSushi_478218171:0,Yum!,FOOD#QUALITY,NaN,0,0,positive
1,en_BlueRibbonSushi_478218171:1,Serves really good sushi.,FOOD#QUALITY,sushi,19,24,positive
2,en_BlueRibbonSushi_478218171:2,Not the biggest portions but adequate.,FOOD#STYLE_OPTIONS,portions,16,24,neutral
3,en_BlueRibbonSushi_478218171:3,Green Tea creme brulee is a must!,FOOD#QUALITY,Green Tea creme brulee,0,22,positive
4,en_BlueRibbonSushi_478218171:4,Don't leave the restaurant without it.,FOOD#QUALITY,NaN,0,0,positive


In [ ]:
# ==============================
# STEP 14: Filter test data for only relevant aspects
# ==============================

# Use the SAME aspect_mapping as train
test_df = test_df[test_df['aspect_category'].isin(aspect_mapping.keys())]

print("Rows after filtering TEST dataset:", len(test_df))
test_df['aspect_category'].value_counts()


Rows after filtering TEST dataset: 742


,count
aspect_category,
FOOD#QUALITY,313
SERVICE#GENERAL,155
RESTAURANT#GENERAL,142
FOOD#STYLE_OPTIONS,55
RESTAURANT#MISCELLANEOUS,33
FOOD#PRICES,23
RESTAURANT#PRICES,21


In [ ]:
# ==============================
# Test data distribution
# ==============================

# Total rows in test set
print("Total rows in TEST dataset:", len(test_df))

# Count of each polarity
print("\nPolarity distribution in TEST dataset:")
print(test_df['polarity'].value_counts())


Total rows in TEST dataset: 742

Polarity distribution in TEST dataset:
polarity
positive    507
negative    196
neutral      39
Name: count, dtype: int64


In [ ]:
# ==============================
# STEP 15: Map aspect_category → simplified aspect
# ==============================

test_df['aspect_simplified'] = test_df['aspect_category'].map(aspect_mapping)

test_df[['aspect_category', 'aspect_simplified']].head()


,aspect_category,aspect_simplified
0,FOOD#QUALITY,FOOD
1,FOOD#QUALITY,FOOD
2,FOOD#STYLE_OPTIONS,FOOD
3,FOOD#QUALITY,FOOD
4,FOOD#QUALITY,FOOD


In [ ]:
# ==============================
# STEP 16: Combine sentence + aspect for model input
# ==============================

test_df['model_input'] = test_df['sentence'] + " [ASP] " + test_df['aspect_simplified']

# Basic text cleaning
test_df['model_input_clean'] = test_df['model_input'].apply(clean_text)

test_df[['sentence', 'aspect_simplified', 'model_input_clean']].head()


,sentence,aspect_simplified,model_input_clean
0,Yum!,FOOD,yum asp food
1,Serves really good sushi.,FOOD,serves really good sushi asp food
2,Not the biggest portions but adequate.,FOOD,not the biggest portions but adequate asp food
3,Green Tea creme brulee is a must!,FOOD,green tea creme brulee is a must asp food
4,Don't leave the restaurant without it.,FOOD,dont leave the restaurant without it asp food


In [ ]:
# ==============================
# STEP 17: Encode polarity labels for TEST
# ==============================

test_df['polarity_encoded'] = test_df['polarity'].map(polarity_mapping)

# Prepare y_test
y_test = test_df['polarity_encoded'].values
y_test[:10]


array([2, 2, 1, 2, 2, 2, 2, 2, 2, 2])

In [ ]:
# ==============================
# STEP 18: Tokenize test data
# ==============================

X_test_seq = tokenizer.texts_to_sequences(test_df['model_input_clean'])
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post', truncating='post')

print("Shape of X_test_pad:", X_test_pad.shape)


Shape of X_test_pad: (742, 50)


In [ ]:
# ==============================
# STEP 19: Import required Keras modules
# ==============================

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
# ==============================
# STEP 20: Hyperparameters
# ==============================

vocab_size = 5000       # Same as tokenizer num_words
embedding_dim = 100     # Size of word embeddings
max_len = 50            # Sequence length (same as padding)
lstm_units = 128        # LSTM neurons
dropout_rate = 0.3      # Dropout for regularization
num_classes = 3         # polarity: positive / neutral / negative
learning_rate = 0.001


In [ ]:
# ==============================
# STEP 21: Build LSTM model
# ==============================

model = Sequential()

# Embedding layer converts word indices → dense vectors
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len))

# Bidirectional LSTM layer
model.add(Bidirectional(LSTM(lstm_units, return_sequences=False)))

# Dropout for regularization
model.add(Dropout(dropout_rate))

# Dense output layer with softmax for 3 classes
model.add(Dense(num_classes, activation='softmax'))

# Compile model
model.compile(
    loss='sparse_categorical_crossentropy',  # suitable for integer labels
    optimizer=Adam(learning_rate=learning_rate),
    metrics=['accuracy']
)

# Model summary
model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# ==============================
# STEP 22: Train model
# ==============================

# Early stopping to prevent overfitting
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    X_train_pad, y_train,
    epochs=10,                 # You can increase later
    batch_size=32,
    validation_split=0.1,      # 10% train → validation
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 38s 290ms/step - accuracy: 0.4717 - loss: 1.0160 - val_accuracy: 0.8408 - val_loss: 0.4671
Epoch 2/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 16s 177ms/step - accuracy: 0.8687 - loss: 0.3762 - val_accuracy: 0.8439 - val_loss: 0.4563
Epoch 3/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 15s 172ms/step - accuracy: 0.9393 - loss: 0.2041 - val_accuracy: 0.9045 - val_loss: 0.2350
Epoch 4/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 17s 187ms/step - accuracy: 0.9509 - loss: 0.1527 - val_accuracy: 0.9108 - val_loss: 0.2213
Epoch 5/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 16s 179ms/step - accuracy: 0.9583 - loss: 0.1277 - val_accuracy: 0.9204 - val_loss: 0.2290
Epoch 6/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 15s 173ms/step - accuracy: 0.9660 - loss: 0.1115 - val_accuracy: 0.9076 - val_loss: 0.2594
Epoch 7/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 16s 177ms/step - accuracy: 0.9598 - loss: 0.1140 - val_accuracy: 0.9140 - val_loss: 0.2403


In [ ]:
# ==============================
# STEP 23: Evaluate model on TEST data
# ==============================

test_loss, test_acc = model.evaluate(X_test_pad, y_test, verbose=1)
print(f"✅ Test Accuracy: {test_acc:.4f}")


24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.7487 - loss: 0.6376
✅ Test Accuracy: 0.7237


In [ ]:
# ==============================
# STEP 24: Predict polarity for new examples
# ==============================

pred_probs = model.predict(X_test_pad)
pred_classes = pred_probs.argmax(axis=1)

# Compare first 10 predictions vs true labels
for i in range(10):
    print(f"Sentence: {test_df['sentence'].iloc[i]}")
    print(f"Aspect: {test_df['aspect_simplified'].iloc[i]}")
    print(f"True: {y_test[i]}, Predicted: {pred_classes[i]}")
    print("-----")


24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step
Sentence: Yum!
Aspect: FOOD
True: 2, Predicted: 2
-----
Sentence: Serves really good sushi.
Aspect: FOOD
True: 2, Predicted: 2
-----
Sentence: Not the biggest portions but adequate.
Aspect: FOOD
True: 1, Predicted: 0
-----
Sentence: Green Tea creme brulee is a must!
Aspect: FOOD
True: 2, Predicted: 2
-----
Sentence: Don't leave the restaurant without it.
Aspect: FOOD
True: 2, Predicted: 2
-----
Sentence: No Comparison
Aspect: DELIVERY_OVERALL
True: 2, Predicted: 0
-----
Sentence: – I can't say enough about this place.
Aspect: DELIVERY_OVERALL
True: 2, Predicted: 2
-----
Sentence: It has great sushi and even better service.
Aspect: FOOD
True: 2, Predicted: 2
-----
Sentence: It has great sushi and even better service.
Aspect: SERVICE
True: 2, Predicted: 2
-----
Sentence: The entire staff was extremely accomodating and tended to my every need.
Aspect: SERVICE
True: 2, Predicted: 2
-----


In [ ]:
# ==============================
# STEP 24: Simple metrics printing
# ==============================

# ==============================
# Overall LSTM Test Performance
# ==============================

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Use pred_classes (not y_pred)
accuracy = accuracy_score(y_test, pred_classes)
precision = precision_score(y_test, pred_classes, average='macro', zero_division=0)
recall = recall_score(y_test, pred_classes, average='macro', zero_division=0)
f1 = f1_score(y_test, pred_classes, average='macro', zero_division=0)

print("✅ LSTM Test Performance")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")





✅ LSTM Test Performance
Accuracy: 0.7237196765498652
Precision: 0.5423885378861405
Recall: 0.512475009727757
F1-score: 0.5175799745820853


In [ ]:
# ==============================
# Predict sentiment for each aspect in a review
# ==============================

def predict_aspect_sentiment_detailed(review, aspects_list):
    """
    Predict sentiment for each aspect in a review using trained LSTM model.

    Args:
        review (str): Full review text
        aspects_list (list): List of aspects to check (e.g., ["FOOD", "SERVICE", "DELIVERY_OVERALL"])

    Returns:
        dict: Each aspect with predicted sentiment and review snippet
    """
    result = {}

    # Map encoded labels back to text
    polarity_map_rev = {0:'Negative', 1:'Neutral', 2:'Positive'}

    for aspect in aspects_list:
        # 1️⃣ Combine sentence + aspect like in training
        input_text = review + " [ASP] " + aspect

        # 2️⃣ Clean text
        input_text_clean = clean_text(input_text)

        # 3️⃣ Tokenize
        seq = tokenizer.texts_to_sequences([input_text_clean])

        # 4️⃣ Pad sequence
        pad_seq = pad_sequences(seq, maxlen=max_len, padding='post')

        # 5️⃣ Predict class
        pred_class = model.predict(pad_seq, verbose=0).argmax(axis=1)[0]

        # 6️⃣ Map to polarity
        predicted_sentiment = polarity_map_rev[pred_class]

        # 7️⃣ Store result
        result[aspect] = {
            "sentiment": predicted_sentiment,
            "snippet": review  # full review as snippet
        }

    return result


In [ ]:
# Example review
review = "burger is yummy serive is bad"

# List of aspects you want to predict
aspects = ["FOOD", "SERVICE", "DELIVERY_OVERALL"]

# Call function with both arguments
result = predict_aspect_sentiment_detailed(review, aspects)

print(result)


{'FOOD': {'sentiment': 'Positive', 'snippet': 'burger is yummy serive is bad'}, 'SERVICE': {'sentiment': 'Positive', 'snippet': 'burger is yummy serive is bad'}, 'DELIVERY_OVERALL': {'sentiment': 'Positive', 'snippet': 'burger is yummy serive is bad'}}


In [ ]:
import pandas as pd
import re

# URL to Pakistani-style reviews Excel
pk_url = "https://github.com/Harjandar/absa-restaurant-sentiment/raw/main/data/raw/pakistani_style_reviews.xlsx"

# Load dataset
pk_df = pd.read_excel(pk_url)

print("✅ Pakistani raw data loaded. Shape:", pk_df.shape)
pk_df.head()


✅ Pakistani raw data loaded. Shape: (130, 2)


,sentence_id,sentence_text
0,1,chana were not cooked properly n very small si...
1,2,Best chicken chilli dry ever had in such a rea...
2,3,no napkins? \n how can you not send any napkin...
3,4,ZINGER BURGER IS VERY VERY POOR IN QUALITY
4,5,very poor delivery of the restaurant. the orde...


In [ ]:
# Inspect columns
print("Columns before renaming:", pk_df.columns.tolist())

# Rename to match SemEval format
pk_df = pk_df.rename(columns={pk_df.columns[0]: "sentence_id",
                              pk_df.columns[1]: "sentence"})

# Drop rows with missing sentences
before_drop = len(pk_df)
pk_df = pk_df.dropna(subset=["sentence"])
after_drop = len(pk_df)
print(f"Rows before drop: {before_drop}, after drop: {after_drop}")


Columns before renaming: ['sentence_id', 'sentence_text']
Rows before drop: 130, after drop: 130


In [ ]:
# Define universal aspects
aspects = ["FOOD", "SERVICE", "DELIVERY_OVERALL"]

# Expand dataset: 1 review → 3 rows (1 per aspect)
pk_rows = []
for _, row in pk_df.iterrows():
    for asp in aspects:
        pk_rows.append({
            "sentence_id": row["sentence_id"],
            "sentence": row["sentence"],
            "aspect_category": asp,    # simplified aspect
            "aspect_term": "N/A",      # unknown
            "from": "",
            "to": "",
            "polarity": "neutral",     # temporary placeholder
            "aspect_simplified": asp
        })

# Create DataFrame
pk_absa = pd.DataFrame(pk_rows)
print("Pakistani dataset after expansion shape:", pk_absa.shape)
pk_absa.head(10)


Pakistani dataset after expansion shape: (390, 8)


,sentence_id,sentence,aspect_category,aspect_term,from,to,polarity,aspect_simplified
0,1,chana were not cooked properly n very small si...,FOOD,N/A,,,neutral,FOOD
1,1,chana were not cooked properly n very small si...,SERVICE,N/A,,,neutral,SERVICE
2,1,chana were not cooked properly n very small si...,DELIVERY_OVERALL,N/A,,,neutral,DELIVERY_OVERALL
3,2,Best chicken chilli dry ever had in such a rea...,FOOD,N/A,,,neutral,FOOD
4,2,Best chicken chilli dry ever had in such a rea...,SERVICE,N/A,,,neutral,SERVICE
5,2,Best chicken chilli dry ever had in such a rea...,DELIVERY_OVERALL,N/A,,,neutral,DELIVERY_OVERALL
6,3,no napkins? \n how can you not send any napkin...,FOOD,N/A,,,neutral,FOOD
7,3,no napkins? \n how can you not send any napkin...,SERVICE,N/A,,,neutral,SERVICE
8,3,no napkins? \n how can you not send any napkin...,DELIVERY_OVERALL,N/A,,,neutral,DELIVERY_OVERALL
9,4,ZINGER BURGER IS VERY VERY POOR IN QUALITY,FOOD,N/A,,,neutral,FOOD


In [ ]:
# Lexicons per aspect (expand for real use)
food_pos = ["acha","achi","achii","tasty","mazedar","delicious","best","good","fantastic"]
food_neg = ["bura","ganda","bad","worst","overcooked","small","not cooked","spoiled"]

service_pos = ["quick","fast","friendly","helpful","attentive"]
service_neg = ["slow","rude","late","bad","worst","unhelpful","delay"]

delivery_pos = ["ontime","good","fast","excellent"]
delivery_neg = ["late","delay","bad","poor","slow"]

# Function to assign polarity per aspect
def assign_aspect_polarity(sentence, aspect):
    sentence = str(sentence).lower()
    polarity = "neutral"
    if aspect == "FOOD":
        if any(word in sentence for word in food_pos):
            polarity = "positive"
        elif any(word in sentence for word in food_neg):
            polarity = "negative"
    elif aspect == "SERVICE":
        if any(word in sentence for word in service_pos):
            polarity = "positive"
        elif any(word in sentence for word in service_neg):
            polarity = "negative"
    elif aspect == "DELIVERY_OVERALL":
        if any(word in sentence for word in delivery_pos):
            polarity = "positive"
        elif any(word in sentence for word in delivery_neg):
            polarity = "negative"
    return polarity

# Apply to Pakistani dataset
pk_absa["polarity"] = pk_absa.apply(
    lambda row: assign_aspect_polarity(row["sentence"], row["aspect_simplified"]),
    axis=1
)

# Map numeric labels
polarity_mapping = {'negative':0, 'neutral':1, 'positive':2}
pk_absa["polarity_encoded"] = pk_absa["polarity"].map(polarity_mapping)

# Check results
print(pk_absa[['sentence','aspect_simplified','polarity','polarity_encoded']].head(20))


                                             sentence aspect_simplified  \
0   chana were not cooked properly n very small si...              FOOD   
1   chana were not cooked properly n very small si...           SERVICE   
2   chana were not cooked properly n very small si...  DELIVERY_OVERALL   
3   Best chicken chilli dry ever had in such a rea...              FOOD   
4   Best chicken chilli dry ever had in such a rea...           SERVICE   
5   Best chicken chilli dry ever had in such a rea...  DELIVERY_OVERALL   
6   no napkins? \n how can you not send any napkin...              FOOD   
7   no napkins? \n how can you not send any napkin...           SERVICE   
8   no napkins? \n how can you not send any napkin...  DELIVERY_OVERALL   
9          ZINGER BURGER IS VERY VERY POOR IN QUALITY              FOOD   
10         ZINGER BURGER IS VERY VERY POOR IN QUALITY           SERVICE   
11         ZINGER BURGER IS VERY VERY POOR IN QUALITY  DELIVERY_OVERALL   
12  very poor delivery of

In [ ]:
# Function to clean text
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()    # remove extra spaces
    return text

# Create model input
pk_absa['model_input'] = pk_absa['sentence'] + " [ASP] " + pk_absa['aspect_simplified']
pk_absa['model_input_clean'] = pk_absa['model_input'].apply(clean_text)

# Check
pk_absa[['sentence','aspect_simplified','model_input_clean','polarity','polarity_encoded']].head(10)


,sentence,aspect_simplified,model_input_clean,polarity,polarity_encoded
0,chana were not cooked properly n very small si...,FOOD,chana were not cooked properly n very small si...,negative,0
1,chana were not cooked properly n very small si...,SERVICE,chana were not cooked properly n very small si...,neutral,1
2,chana were not cooked properly n very small si...,DELIVERY_OVERALL,chana were not cooked properly n very small si...,neutral,1
3,Best chicken chilli dry ever had in such a rea...,FOOD,best chicken chilli dry ever had in such a rea...,positive,2
4,Best chicken chilli dry ever had in such a rea...,SERVICE,best chicken chilli dry ever had in such a rea...,neutral,1
5,Best chicken chilli dry ever had in such a rea...,DELIVERY_OVERALL,best chicken chilli dry ever had in such a rea...,neutral,1
6,no napkins? \n how can you not send any napkin...,FOOD,no napkins how can you not send any napkins ev...,neutral,1
7,no napkins? \n how can you not send any napkin...,SERVICE,no napkins how can you not send any napkins ev...,neutral,1
8,no napkins? \n how can you not send any napkin...,DELIVERY_OVERALL,no napkins how can you not send any napkins ev...,negative,0
9,ZINGER BURGER IS VERY VERY POOR IN QUALITY,FOOD,zinger burger is very very poor in quality asp...,neutral,1


In [ ]:
# train_df = your original cleaned & balanced TRAIN dataset
print("Original TRAIN shape:", train_df.shape)
print("Pakistani dataset shape:", pk_absa.shape)

# Combine
train_combined = pd.concat([train_df, pk_absa], ignore_index=True)
print("✅ Combined TRAIN dataset shape:", train_combined.shape)

# Check polarity distribution
print(train_combined['polarity'].value_counts())


Original TRAIN shape: (3132, 11)
Pakistani dataset shape: (390, 11)
✅ Combined TRAIN dataset shape: (3522, 12)
polarity
neutral     1323
positive    1124
negative    1075
Name: count, dtype: int64


In [ ]:
# ==============================
# STEP 30b: Print Pakistani dataset with assigned labels
# ==============================

# Show first 20 rows with relevant columns
print(pk_absa[['sentence_id', 'sentence', 'aspect_simplified', 'polarity', 'polarity_encoded']].head(20))

# Optional: View full dataset (careful if large!)
# pd.set_option('display.max_rows', None)
# print(pk_absa[['sentence_id', 'sentence', 'aspect_simplified', 'polarity', 'polarity_encoded']])


    sentence_id                                           sentence  \
0             1  chana were not cooked properly n very small si...   
1             1  chana were not cooked properly n very small si...   
2             1  chana were not cooked properly n very small si...   
3             2  Best chicken chilli dry ever had in such a rea...   
4             2  Best chicken chilli dry ever had in such a rea...   
5             2  Best chicken chilli dry ever had in such a rea...   
6             3  no napkins? \n how can you not send any napkin...   
7             3  no napkins? \n how can you not send any napkin...   
8             3  no napkins? \n how can you not send any napkin...   
9             4         ZINGER BURGER IS VERY VERY POOR IN QUALITY   
10            4         ZINGER BURGER IS VERY VERY POOR IN QUALITY   
11            4         ZINGER BURGER IS VERY VERY POOR IN QUALITY   
12            5  very poor delivery of the restaurant. the orde...   
13            5  ver

In [ ]:
# ==============================
# STEP 31: Merge Pakistani ABSA with original TRAIN dataset
# ==============================

print("Original TRAIN shape:", train_df.shape)
print("Pakistani ABSA shape:", pk_absa.shape)

train_combined = pd.concat([train_df, pk_absa], ignore_index=True)
print("✅ Combined TRAIN dataset shape:", train_combined.shape)

# Check polarity distribution
print("Polarity distribution in combined dataset:")
print(train_combined['polarity'].value_counts())


Original TRAIN shape: (3132, 11)
Pakistani ABSA shape: (390, 11)
✅ Combined TRAIN dataset shape: (3522, 12)
Polarity distribution in combined dataset:
polarity
neutral     1323
positive    1124
negative    1075
Name: count, dtype: int64


In [ ]:
# ==============================
# STEP X: UPSAMPLING MINORITY CLASSES
# ==============================

from sklearn.utils import resample

# Separate classes using correct dataframe
df_neg = train_combined[train_combined['polarity_encoded'] == 0]
df_neu = train_combined[train_combined['polarity_encoded'] == 1]
df_pos = train_combined[train_combined['polarity_encoded'] == 2]

print("Before balancing:")
print(df_neg.shape, df_neu.shape, df_pos.shape)

# Find maximum class size
max_size = max(len(df_neg), len(df_neu), len(df_pos))

# Upsample each class
df_neg_up = resample(df_neg, replace=True, n_samples=max_size, random_state=42)
df_neu_up = resample(df_neu, replace=True, n_samples=max_size, random_state=42)
df_pos_up = resample(df_pos, replace=True, n_samples=max_size, random_state=42)

# Combine balanced dataset
train_balanced = pd.concat([df_neg_up, df_neu_up, df_pos_up])

# Shuffle dataset
train_balanced = train_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

print("\n✅ After balancing:")
print(train_balanced['polarity_encoded'].value_counts())


Before balancing:
(1075, 12) (1323, 12) (1124, 12)

✅ After balancing:
polarity_encoded
1    1323
2    1323
0    1323
Name: count, dtype: int64


In [ ]:
# ==============================
# STEP 32: Tokenization for combined dataset
# ==============================

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_words = 5000
max_len = 50

# Fit tokenizer on combined dataset
tokenizer_combined = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer_combined.fit_on_texts(train_combined['model_input_clean'])

# Convert to sequences
X_train_seq = tokenizer_combined.texts_to_sequences(train_combined['model_input_clean'])
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')

print("Shape of combined padded sequences:", X_train_pad.shape)

# Prepare labels
y_train = train_combined['polarity_encoded'].values


Shape of combined padded sequences: (3522, 50)


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_words = 5000  # vocabulary size
max_len = 50      # sequence length

# Initialize tokenizer
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(train_combined['model_input_clean'])

# Convert texts to sequences
X_train_seq = tokenizer.texts_to_sequences(train_combined['model_input_clean'])

# Pad sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')

print("Example tokenized + padded sequence shape:", X_train_pad.shape)
print("Example sequence:", X_train_pad[0])


Example tokenized + padded sequence shape: (3522, 50)
Example sequence: [ 19  25 226 142  85 535   2  14   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]


In [ ]:
import numpy as np

# Labels
y_train = train_combined['polarity_encoded'].values
print("Example labels:", y_train[:20])
print("Shape of labels:", y_train.shape)


Example labels: [0 1 2 2 0 1 1 0 0 2 0 1 2 2 2 1 0 1 1 2]
Shape of labels: (3522,)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Hyperparameters
embedding_dim = 100
lstm_units = 128
dropout_rate = 0.3
num_classes = 3
learning_rate = 0.001

# Build model
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
model.add(Bidirectional(LSTM(lstm_units)))
model.add(Dropout(dropout_rate))
model.add(Dense(num_classes, activation='softmax'))

# Compile
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(learning_rate=learning_rate),
    metrics=['accuracy']
)

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    X_train_pad, y_train,
    epochs=10,           # can increase for better learning
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 23s 175ms/step - accuracy: 0.4735 - loss: 1.0049 - val_accuracy: 0.1671 - val_loss: 2.3935
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 18s 175ms/step - accuracy: 0.8636 - loss: 0.3835 - val_accuracy: 0.2153 - val_loss: 2.9275
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 18s 180ms/step - accuracy: 0.9410 - loss: 0.1980 - val_accuracy: 0.2323 - val_loss: 2.5904
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 17s 170ms/step - accuracy: 0.9509 - loss: 0.1585 - val_accuracy: 0.2776 - val_loss: 2.4641


In [ ]:
# We already prepared test dataset (cleaned & tokenized)
# X_test_pad, y_test

test_loss, test_acc = model.evaluate(X_test_pad, y_test, verbose=1)
print(f"✅ Test Accuracy on original SemEval test set: {test_acc:.4f}")


24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.4240 - loss: 1.1455
✅ Test Accuracy on original SemEval test set: 0.4016


In [ ]:
def predict_aspect_sentiment(review, aspects_list):
    """
    Predict sentiment for each aspect in a mixed English + Roman Urdu review.
    """
    polarity_map_rev = {0:'Negative', 1:'Neutral', 2:'Positive'}
    result = {}
    for aspect in aspects_list:
        text = review + " [ASP] " + aspect
        seq = tokenizer.texts_to_sequences([clean_text(text)])
        pad_seq = pad_sequences(seq, maxlen=max_len, padding='post')
        pred_class = model.predict(pad_seq, verbose=0).argmax(axis=1)[0]
        result[aspect] = polarity_map_rev[pred_class]
    return result

# Test example
review = "biryani bahut achii thi but service is very slow"
aspects = ["FOOD", "SERVICE", "DELIVERY_OVERALL"]

pred_result = predict_aspect_sentiment(review, aspects)
print("Prediction for mixed Roman + English review:")
print(pred_result)


Prediction for mixed Roman + English review:
{'FOOD': 'Negative', 'SERVICE': 'Negative', 'DELIVERY_OVERALL': 'Negative'}


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Predict all test data
pred_probs = model.predict(X_test_pad)
pred_classes = pred_probs.argmax(axis=1)

accuracy = accuracy_score(y_test, pred_classes)
precision = precision_score(y_test, pred_classes, average='macro', zero_division=0)
recall = recall_score(y_test, pred_classes, average='macro', zero_division=0)
f1 = f1_score(y_test, pred_classes, average='macro', zero_division=0)

print("✅ LSTM Test Performance (Original TEST set)")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")


24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step
✅ LSTM Test Performance (Original TEST set)
Accuracy: 0.4016
Precision: 0.3751
Recall: 0.3750
F1-score: 0.3050
